In [1]:
import numpy as np
import matplotlib.pyplot as plt
import rawpy

In [2]:
raw = rawpy.imread("02.CR3")
array = np.array(raw.raw_image_visible)
array = array.astype(np.int64)

In [3]:
array

array([[1131, 1930, 1100, ...,  685,  607,  675],
       [1920, 1604, 1836, ...,  636,  669,  638],
       [1139, 1851, 1124, ...,  704,  599,  662],
       ...,
       [1393, 1178, 1380, ...,  681,  804,  695],
       [ 889, 1345,  940, ...,  752,  645,  791],
       [1347, 1155, 1373, ...,  682,  766,  684]], dtype=int64)

In [4]:
array.max(), array.min()

(16383, 522)

In [5]:
import rawpy
import numpy as np

# File paths for your CR3 files
cr3_files = ["00.CR3", "01.CR3", "02.CR3", "03.CR3","04.CR3", "05.CR3","06.CR3", "07.CR3","08.CR3", "09.CR3", "10.CR3"]

In [6]:
cr3_files[1:]

['01.CR3',
 '02.CR3',
 '03.CR3',
 '04.CR3',
 '05.CR3',
 '06.CR3',
 '07.CR3',
 '08.CR3',
 '09.CR3',
 '10.CR3']

In [7]:
from tqdm.auto import tqdm

In [8]:
raw = rawpy.imread("00.CR3")
h = np.array(raw.raw_image_visible).astype(np.int64)

k=1
for file in cr3_files[1:]:
    raw = rawpy.imread(file)
    i = np.array(raw.raw_image_visible).astype(np.int64)
    i = np.array(i) * (2**k)
    t = 0.8 * h.max()
    mask = h > t
    h[mask] = i[mask]
    k+=1


In [9]:
h.shape

(4660, 6984)

In [10]:
import cv2

In [11]:
cv2.imwrite("HDR_original.png", h)

True

In [12]:
h.max()

2008064

In [13]:
h.min()

598

### Apply the demosaicing algorithm

In [14]:
import numpy as np
import cv2

def bilinear_demosaicing(raw_data):
    # Assuming raw_data is a numpy array of the RAW image data

    # Height and width of the raw image
    height, width = raw_data.shape

    # Create an empty image for the demosaiced output
    # This will be a 3-channel image for the RGB color space
    demosaiced_img = np.zeros((height, width, 3), dtype=np.int64)

    # Bilinear demosaicing algorithm
    for row in tqdm(range(1, height - 1)):
        for col in range(1, width - 1):
            if (row % 2 == 1) and (col % 2 == 1):  # Blue pixel
                demosaiced_img[row, col, 0] = raw_data[row, col]  # Blue
                demosaiced_img[row, col, 1] = (raw_data[row, col - 1] + raw_data[row, col + 1] +
                                               raw_data[row - 1, col] + raw_data[row + 1, col]) // 4  # Green
                demosaiced_img[row, col, 2] = (raw_data[row - 1, col - 1] + raw_data[row - 1, col + 1] +
                                               raw_data[row + 1, col - 1] + raw_data[row + 1, col + 1]) // 4  # Red
            elif (row % 2 == 0) and (col % 2 == 0):  # Red pixel
                demosaiced_img[row, col, 0] = (raw_data[row - 1, col - 1] + raw_data[row - 1, col + 1] +
                                               raw_data[row + 1, col - 1] + raw_data[row + 1, col + 1]) // 4  # Blue
                demosaiced_img[row, col, 1] = (raw_data[row, col - 1] + raw_data[row, col + 1] +
                                               raw_data[row - 1, col] + raw_data[row + 1, col]) // 4  # Green
                demosaiced_img[row, col, 2] = raw_data[row, col]  # Red
            else:  # Green pixel
                demosaiced_img[row, col, 1] = raw_data[row, col]  # Green
                if row % 2 == 0:
                    demosaiced_img[row, col, 0] = (raw_data[row - 1, col] + raw_data[row + 1, col]) // 2  # Blue
                    demosaiced_img[row, col, 2] = (raw_data[row, col - 1] + raw_data[row, col + 1]) // 2  # Red
                     
                else:
                    demosaiced_img[row, col, 0] = (raw_data[row, col - 1] + raw_data[row, col + 1]) // 2  # Blue
                    demosaiced_img[row, col, 2] = (raw_data[row - 1, col] + raw_data[row + 1, col]) // 2  # Red


    return demosaiced_img


demosaiced_image = bilinear_demosaicing(h)

  0%|          | 0/4658 [00:00<?, ?it/s]

In [15]:
cv2.imwrite("HDR_demosaiced_image.png", demosaiced_image)

True

In [16]:
demosaiced_image[:,:,2]

array([[   0,    0,    0, ...,    0,    0,    0],
       [   0, 3014, 3026, ...,  888,  888,    0],
       [   0, 3027, 3063, ...,  879,  882,    0],
       ...,
       [   0, 2171, 2213, ..., 1088, 1098,    0],
       [   0, 2125, 2145, ..., 1050, 1082,    0],
       [   0,    0,    0, ...,    0,    0,    0]], dtype=int64)

In [17]:
demosaiced_image = demosaiced_image[1:-1, 1:-1]

In [18]:
demosaiced_image[:,:,2]

array([[3014, 3026, 3028, ...,  888,  888,  888],
       [3027, 3063, 3050, ...,  876,  879,  882],
       [3020, 3044, 3052, ...,  883,  877,  870],
       ...,
       [2217, 2281, 2300, ..., 1138, 1126, 1114],
       [2171, 2213, 2263, ..., 1078, 1088, 1098],
       [2125, 2145, 2226, ..., 1019, 1050, 1082]], dtype=int64)

In [19]:
demosaiced_image.min()

598

In [20]:
demosaiced_image.max()

2008064

In [21]:
demosaiced_image.shape

(4658, 6982, 3)

### Applying White balance

In [22]:
def apply_white_balance(img):
    # Calculate the average color of each RGB channel
    avg_r = np.mean(img[:,:,2])
    avg_g = np.mean(img[:,:,1])
    avg_b = np.mean(img[:,:,0])
    # Calculate the overall average color
    avg_gray = (avg_r + avg_g + avg_b) / 3

    # Scale each channel
    img[:,:,2] = (img[:,:,2] / avg_r) * avg_gray
    img[:,:,1] = (img[:,:,1] / avg_g) * avg_gray
    img[:,:,0] = (img[:,:,0] / avg_b) * avg_gray

    return img

# Apply white balance
white_balanced_image = apply_white_balance(demosaiced_image)

# Save the white balanced image
white_balanced_output_path = 'HDR_demosaiced_white_balance.png'
cv2.imwrite(white_balanced_output_path, white_balanced_image)

True

### Decreasing the dynamic range by computing the logarithm of this data

In [23]:
log_image = np.log(white_balanced_image)

# Checking the new maximum and minimum values in the transformed image
new_max = log_image.max()
new_min = log_image.min()

new_max, new_min

(14.200553887141437, 6.236369590203704)

### Downscaling it to the [0, 255] interval

In [24]:
def map_to_255(image):
    # Assuming the image is in some range and you want to scale it to 0-255
    min_val = np.min(image)
    max_val = np.max(image)
    scaled_image = ((image - min_val) / (max_val - min_val)) * 255
    return scaled_image.astype(np.uint8)

demosaiced_image = map_to_255(log_image)

In [25]:
demosaiced_image.min()

0

In [26]:
demosaiced_image.max()

255

In [27]:
output_path = 'HDR_demosaiced_image_normalized_white_balance.png'
cv2.imwrite(output_path, demosaiced_image)

True

### iCAM06

In [28]:
def icam06_tone_mapping(image, output_range=4):
    # Convert image to float
    image = image.astype(np.float32)

    # Calculate input_intensity
    input_intensity = (20 * image[:, :, 2] + 40 * image[:, :, 1] + image[:, :, 0]) / 61

    # Normalize RGB values
    r, g, b = image[:, :, 2] / input_intensity, image[:, :, 1] / input_intensity, image[:, :, 0] / input_intensity

    # Compute log_base using bilateral filter
    log_base = cv2.bilateralFilter(np.log(input_intensity), d=5, sigmaColor=75, sigmaSpace=75)

    # Calculate log_details
    log_details = np.log(input_intensity) - log_base

    # Compression calculation
    compression = np.log(output_range) / (np.max(log_base) - np.min(log_base))

    # Offset calculation
    log_offset = -np.max(log_base) * compression

    # Calculate output_intensity
    output_intensity = np.exp(log_base * compression + log_offset + log_details)

    # Apply output intensity to RGB values
    processed_image = np.stack([r, g, b], axis=-1) * output_intensity[:, :, None]

    # Clip values to [0, 1] and convert to 8-bit format
    processed_image = np.clip(processed_image, 0, 1) * 255
    return processed_image.astype(np.uint8)

result = icam06_tone_mapping(demosaiced_image)

white_balanced_output_path = 'HDR_demosaiced_image_normalized_white_balance_ICAM06.png'
cv2.imwrite(white_balanced_output_path, result)

True